In [1]:
import numpy as  np
import codecs
import matplotlib.pyplot as plt
import unicodedata
import re
from collections import Counter,defaultdict
import nltk.corpus.reader as pt
import os

import string
import pdb
import nltk
import scipy

import sklearn

In [2]:
def readAFile(nf):
    f = open(nf, 'rb')
    l = []
    txt = f.readlines()
    for i in txt:
        l.append(i.decode("utf-8"))
    f.close()
    return l

In [3]:
def process(txt):
    #txt = txt[txt.find("\n\n"):] # elimination de l'entete (on ne conserve que les caractères après la première occurence du motif
    txt = unicodedata.normalize("NFKD",txt).encode("ascii","ignore") # elimination des caractères spéciaux, accents...

    punc = string.punctuation    # recupération de la ponctuation
    punc += u'\n\r\t\\'          # ajouts de caractères à enlever
    table =string.maketrans(punc, ' '*len(punc))  # table de conversion punc -> espace
    txt = string.translate(txt,table).lower() # elimination des accents + minuscules
    return txt
    #return re.sub(" +"," ", txt) # expression régulière pour transformer les espaces multiples en simples espaces


In [4]:
t = readAFile('train.utf8')
x=[]
y=[]
for txt in t:
    lab = re.sub(r"<[0-9]*:[0-9]*:(.)>.*","\\1",txt)
    txt = re.sub(r"<[0-9]*:[0-9]*:.>(.*)","\\1",txt)
    x.append(txt)
    #print lab
    if('C' in lab):
        bin_lab=1
    elif('M' in lab):
        bin_lab=0
    else:
        bin_lab='err'
    y.append(bin_lab)
x=[process(xi) for xi in x]
print x[0][0]

In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer

stopList = []

# On instancie le tokenizer
tokenizer = WordPunctTokenizer()
french_stopwords = set(stopwords.words('french'))

for a in range(len(x)):

    # L'interface étant identique, le reste du code est le même
    tokens = tokenizer.tokenize(x[a])


    # chargement des stopwords français

    # un petit filtre
    tokens = [token for token in tokens if token.lower() not in french_stopwords]

    stopList.append(' '.join(tokens))


In [6]:
stopList[0]

'quand dis chers amis agit formule diplomatique expression ressens'

In [6]:

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation



countVe = CountVectorizer(strip_accents='unicode',max_df=0.95, min_df=1, analyzer="word",ngram_range=(1, 3) )

tfve = TfidfVectorizer(strip_accents='unicode' ,max_df=0.9, min_df=1, analyzer="word",ngram_range=(1, 2) )


In [7]:



# count = countVe.fit_transform(stopList)
idf = tfve.fit_transform(stopList)



In [175]:
idf

<57413x405568 sparse matrix of type '<type 'numpy.float64'>'
	with 1314704 stored elements in Compressed Sparse Row format>

In [17]:
import numpy as np
import sklearn.naive_bayes as nb
from sklearn import svm
from sklearn import linear_model as lin

# données ultra basiques, à remplacer par vos corpus vectorisés
X = idf
# X = count
y = np.array(y)

# SVM
clf = svm.LinearSVC(C=.8,class_weight={0:1.4,1:1}, max_iter=1000000)
# apprentissage
clf.fit(X, y)  

LinearSVC(C=0.8, class_weight={0: 1.4, 1: 1}, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [18]:
t = readAFile('test.utf8')
test=[]
for txt in t:
    
    txt = re.sub(r"<[0-9]*:[0-9]*:.>(.*)","\\1",txt)
    test.append(txt)    

test=[process(xi) for xi in test]

testList = []

# On instancie le tokenizer
tokenizer = WordPunctTokenizer()
french_stopwords = set(stopwords.words('french'))

for a in range(len(test)):

    # L'interface étant identique, le reste du code est le même
    tokens = tokenizer.tokenize(test[a])


    # chargement des stopwords français

    # un petit filtre
    tokens = [token for token in tokens if token.lower() not in french_stopwords]

    testList.append(' '.join(tokens))


In [29]:
print len(testList)
a = tfve.transform(test)

27162


In [33]:
pp = clf.predict(a) 

In [28]:
for i in range(2,len(pp)-2):
    if pp[i-1] == pp[i+1]:
        pp[i] = pp[i-1]
for i in range(2,len(pp)-2):
    if pp[i-2] == pp[i+2]:
        pp[i] = pp[i-2]
# for i in range(3,len(pp)-3):
#     if pp[i-3] == pp[i+3]:
#         pp[i] = pp[i-3]        

In [34]:
def post(y,rang,coeff):
    y_new=np.zeros((len(y)))
    for i in np.random.permutation(range(len(y))):
        findex=i-rang
        lindex=i+rang
        #print 'f'+str(findex)
        #print 'l'+str(lindex)
        if(i-rang<0):
            findex=0
            lindex=rang+i
        elif(i+rang>len(y)):
            findex=i-rang
            lindex=len(y)-1
        countM=len(np.where(y[findex:lindex]==0)[0])
        countC=len(np.where(y[findex:lindex]==1)[0])
        y_new[i]=np.argmax([countM*coeff,countC])
    return y_new
pp=post(post(pp,5,2.45),5,1.)

In [35]:


f = open('svm.txt', 'w')
for i in pp:
    if i == 1:
        f.write('C\n')
    else:
        f.write('M\n')
f.close()

In [14]:
len(pp)

27162